In [ ]:
!python3 -m vllm.entrypoints.api_server --host "0.0.0.0" --model TheBloke/Xwin-LM-13B-V0.1-AWQ --quantization awq

INFO 09-22 07:05:45 llm_engine.py:72] Initializing an LLM engine with config: model='TheBloke/Xwin-LM-13B-V0.1-AWQ', tokenizer='TheBloke/Xwin-LM-13B-V0.1-AWQ', tokenizer_mode=auto, revision=None, trust_remote_code=False, dtype=torch.float16, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)
INFO 09-22 07:11:17 llm_engine.py:202] # GPU blocks: 1074, # CPU blocks: 327
INFO:     Started server process [80]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO 09-22 07:11:33 async_llm_engine.py:371] Received request 61a7080114e348b29d8cae2138b4163b: prompt: '富士山について教えて', sampling params: SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, temperature=1.0, top_p=1.0, top_k=-1, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], ignore_eos=False, max_tokens=256, logprobs=None), prompt token ids: None.
INFO

In [1]:
%%time
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/Xwin-LM-13B-V0.1-AWQ" # or 7B

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, # fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)

def chat(prompt, max_new_tokens):
    tokens = tokenizer(
        prompt,
        return_tensors='pt'
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=max_new_tokens
    )
    return tokenizer.decode(generation_output[0])

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Replacing layers...: 100% 40/40 [00:03<00:00, 10.92it/s]


In [4]:
%%time
prompt = """日本の観光地の平均気温をjsonで書き出すと、"""
print(chat(prompt, 256))

<s> 日本の観光地の平均気温をjsonで書き出すと、以下のようなコードがあります。
```javascript
const axios = require('axios');

const url = 'https://api.ipapi.co/2/json/24/all/weather/sunshine/japan/';
const filePath = 'japan_sunshine.json';

axios.get(url)
  .then(response => {
    const data = response.data.data.list;
    const output = data.map(item => {
      const temp = item.main.temp_max;
      const sunshine = item.weather[0].sun.sunshine;
      const weather = item.weather[0].main;
      return {
        city: item.city.name_ja,
        temperature: `${temp}℃`,
        sunshine: `${sunshine}%`,
        weather: `${weather}`,
      };
    });
    fs.writeFileSync(filePath, JSON.stringify(output, null, 2));
    console.log('Data saved to',
CPU times: user 4.09 s, sys: 15.7 ms, total: 4.1 s
Wall time: 4.1 s


In [6]:
%%time
prompt = """小学生向けに計算問題(2桁の四則演算)を5問出力してください。まず問題を並べて、そのあとに解答のみを並べてください。

"""
print(chat(prompt, 256))

<s> 小学生向けに計算問題(2桁の四則演算)を5問出力してください。まず問題を並べて、そのあとに解答のみを並べてください。

問題:
1. 1 + 2 = ?
2. 3 - 4 = ?
3. 6 \* 2 = ?
4. 9 / 3 = ?
5. 4 \* 5 = ?

解答:
1. 3
2. -1
3. 12
4. 3
5. 20</s>
CPU times: user 1.58 s, sys: 1.15 ms, total: 1.58 s
Wall time: 1.58 s
